##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Recomendando filmes: modelos de recomendação no TFX

Observação: recomendamos executar este tutorial em um notebook Colab, sem necessidade de configuração! Basta clicar em “Executar no Google Colab”.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/recommenders"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/recommenders.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/recommenders.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a></td>
<td>     <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/tfx/recommenders.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table></div>

## Tutorial TFRS portado para o TFX

Esta é uma versão de um tutorial básico do TensorFlow Recommenders (TFRS) para o TFX, que foi projetado para demonstrar como usar o TFRS em um pipeline do TFX. Ele reflete o [tutorial básico](https://www.tensorflow.org/recommenders/examples/basic_retrieval).

Para contextualizar, os sistemas de recomendação do mundo real são frequentemente compostos de duas fases:

1. A fase de recuperação é responsável por selecionar um conjunto inicial de centenas de candidatos dentre todos os candidatos possíveis. O principal objetivo deste modelo é eliminar de forma eficiente todos os candidatos nos quais o usuário não está interessado. Como o modelo de recuperação pode lidar com milhões de candidatos, ele deve ser computacionalmente eficiente.
2. O estágio de classificação pega os resultados do modelo de recuperação e os ajusta para selecionar o melhor conjunto possível de recomendações. Sua tarefa é restringir o conjunto de itens nos quais o usuário pode estar interessado numa lista restrita de prováveis ​​candidatos.

Neste tutorial, vamos nos concentrar no primeiro estágio, a recuperação. Os modelos de recuperação são frequentemente compostos por dois submodelos:

1. Um modelo de consulta que calcula a representação da consulta (normalmente um vetor de embedding de dimensionalidade fixa) usando características de consulta.
2. Um modelo candidato que calcula a representação candidata (um vetor de tamanho igual) usando as características candidatas

As saídas dos dois modelos são então multiplicadas para fornecer uma pontuação de afinidade consulta-candidato, com pontuações mais altas expressando uma melhor correspondência entre o candidato e a consulta.

Neste tutorial, construiremos e treinaremos esse modelo de duas torres usando o dataset Movielens.

Nós iremos:

1. Ingerir e inspecionar o dataset MovieLens.
2. Implementar um modelo de recuperação.
3. Treinar e exportar o modelo.
4. Fazer previsões

## O dataset

O dataset Movielens é um dataset clássico do grupo de pesquisa [GroupLens](https://grouplens.org/datasets/movielens/) da Universidade de Minnesota. Ele contém um conjunto de classificações dadas a filmes por um conjunto de usuários e é um carro-chefe de pesquisa para sistemas de recomendação.

Os dados podem ser tratados de duas maneiras:

1. Podem ser interpretados como uma expressão de quais filmes os usuários assistiram (e avaliaram) e quais não. Esta é uma forma de feedback implícito, onde os filmes que os usuários assistiram vão nos dizer quais coisas eles preferem ver e quais preferem não ver.
2. Também podem ser vistos como uma expressão do quanto os usuários gostaram dos filmes que assistiram. Esta é uma forma de feedback explícito: dado que um usuário assistiu a um filme, podemos dizer aproximadamente o quanto ele gostou com base na avaliação que deu.

Neste tutorial, estamos focando num sistema de recuperação: um modelo que prevê um conjunto de filmes do catálogo que o usuário provavelmente assistirá. Freqüentemente, os dados implícitos são mais úteis aqui e, portanto, trataremos o Movielens como um sistema implícito. Isto significa que todo filme que um usuário assistiu é um exemplo positivo e todo filme que ele não viu é um exemplo negativo implícito.

## Importações

Vamos primeiro tirar nossas importações do caminho.

In [ ]:
!pip install -Uq tfx
!pip install -Uq tensorflow-recommenders
!pip install -Uq tensorflow-datasets

### Desinstale o shapely

TODO(b/263441833) Esta é uma solução temporária para evitar um ImportError. Em última análise, isto deverá ser resolvido com suporte a uma versão mais recente do Bigquery, em vez de desinstalar outras dependências extras.

In [ ]:
!pip uninstall shapely -y

### Você reiniciou o runtime?

Se você estiver usando o Google Colab, na primeira vez que executar a célula acima, você deve reiniciar o runtime ("Runtime &gt; Restart runtime ..."). Isso é necessário devido à maneira como o Colab carrega os pacotes.

In [ ]:
import os
import absl
import json
import pprint
import tempfile

from typing import Any, Dict, List, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import apache_beam as beam

from absl import logging

from tfx.components.example_gen.base_example_gen_executor import BaseExampleGenExecutor
from tfx.components.example_gen.component import FileBasedExampleGen
from tfx.components.example_gen import utils
from tfx.dsl.components.base import executor_spec

from tfx.types import artifact
from tfx.types import artifact_utils
from tfx.types import channel
from tfx.types import standard_artifacts
from tfx.types.standard_artifacts import Examples

from tfx.dsl.component.experimental.annotations import InputArtifact
from tfx.dsl.component.experimental.annotations import OutputArtifact
from tfx.dsl.component.experimental.annotations import Parameter
from tfx.dsl.component.experimental.decorators import component
from tfx.types.experimental.simple_artifacts import Dataset

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Set up logging.
tf.get_logger().propagate = False
absl.logging.set_verbosity(absl.logging.INFO)
pp = pprint.PrettyPrinter()

print(f"TensorFlow version: {tf.__version__}")
print(f"TFX version: {tfx.__version__}")
print(f"TensorFlow Recommenders version: {tfrs.__version__}")

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

## Crie um TFDS ExampleGen

Criamos um [componente ExampleGen personalizado](https://www.tensorflow.org/tfx/guide/examplegen#custom_examplegen) que usamos para carregar um dataset TensorFlow Datasets (TFDS). Isso usa um executor personalizado em um FileBasedExampleGen.

In [ ]:
@beam.ptransform_fn
@beam.typehints.with_input_types(beam.Pipeline)
@beam.typehints.with_output_types(tf.train.Example)
def _TFDatasetToExample(  # pylint: disable=invalid-name
    pipeline: beam.Pipeline,
    exec_properties: Dict[str, Any],
    split_pattern: str
    ) -> beam.pvalue.PCollection:
    """Read a TensorFlow Dataset and create tf.Examples"""
    custom_config = json.loads(exec_properties['custom_config'])
    dataset_name = custom_config['dataset']
    split_name = custom_config['split']

    builder = tfds.builder(dataset_name)
    builder.download_and_prepare()

    return (pipeline
            | 'MakeExamples' >> tfds.beam.ReadFromTFDS(builder, split=split_name)
            | 'AsNumpy' >> beam.Map(tfds.as_numpy)
            | 'ToDict' >> beam.Map(dict)
            | 'ToTFExample' >> beam.Map(utils.dict_to_example)
            )

class TFDSExecutor(BaseExampleGenExecutor):
  def GetInputSourceToExamplePTransform(self) -> beam.PTransform:
    """Returns PTransform for TF Dataset to TF examples."""
    return _TFDatasetToExample

## Inicialize o contexto inicial do pipeline TFX

In [ ]:
context = InteractiveContext()

## Preparando o dataset

Usaremos nosso executor personalizado em `FileBasedExampleGen` para carregar nossos datasets TFDS. Como temos dois datasets, criaremos dois componentes `ExampleGen`.

In [ ]:
# Ratings data.
ratings_example_gen = FileBasedExampleGen(
    input_base='dummy',
    custom_config={'dataset':'movielens/100k-ratings', 'split':'train'},
    custom_executor_spec=executor_spec.ExecutorClassSpec(TFDSExecutor))
context.run(ratings_example_gen, enable_cache=True)

In [ ]:
# Features of all the available movies.
movies_example_gen = FileBasedExampleGen(
    input_base='dummy',
    custom_config={'dataset':'movielens/100k-movies', 'split':'train'},
    custom_executor_spec=executor_spec.ExecutorClassSpec(TFDSExecutor))
context.run(movies_example_gen, enable_cache=True)

## Crie o utilitário `inspect_examples`

Criamos um utilitário conveniente para inspecionar datasets de TF.Examples. O dataset de classificações retorna um dicionário com ID do filme, ID do usuário, classificação atribuída, carimbo de data/hora, informações do filme e informações do usuário:

In [ ]:
def inspect_examples(component,
                     channel_name='examples',
                     split_name='train',
                     num_examples=1):
  # Get the URI of the output artifact, which is a directory
  full_split_name = 'Split-{}'.format(split_name)
  print('channel_name: {}, split_name: {} (\"{}\"), num_examples: {}\n'.format(
      channel_name, split_name, full_split_name, num_examples))
  train_uri = os.path.join(
      component.outputs[channel_name].get()[0].uri, full_split_name)

  # Get the list of files in this directory (all compressed TFRecord files)
  tfrecord_filenames = [os.path.join(train_uri, name)
                        for name in os.listdir(train_uri)]

  # Create a `TFRecordDataset` to read these files
  dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

  # Iterate over the records and print them
  for tfrecord in dataset.take(num_examples):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

inspect_examples(ratings_example_gen)

O dataset de filmes contém a identificação do filme, o título do filme e dados sobre os gêneros aos quais ele pertence. Observe que os gêneros são codificados com rótulos de valor inteiro.

In [ ]:
inspect_examples(movies_example_gen)

## O ExampleGen fez a divisão

Quando ingerimos o dataset Movielens, nosso componente `ExampleGen` dividiu os dados em divisões `train` e `eval`. Na verdade, eles são chamados de `Split-train` e `Split-eval`. Por padrão, a divisão é 66% para treinamento e 34% para avaliação.

## Gere estatísticas para filmes e classificações

Para um pipeline TFX, precisamos gerar estatísticas para o dataset. Fazemos isso usando um [componente StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen). Eles serão usados ​​pelo [componente SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen) abaixo quando gerarmos um esquema para nosso dataset. De qualquer forma, esta é uma boa prática, porque é importante examinar e analisar seus dados continuamente. Como temos dois datasets, criaremos dois componentes StatisticsGen.

In [ ]:
movies_stats_gen = tfx.components.StatisticsGen(
    examples=movies_example_gen.outputs['examples'])
context.run(movies_stats_gen, enable_cache=True)

In [ ]:
context.show(movies_stats_gen.outputs['statistics'])

In [ ]:
ratings_stats_gen = tfx.components.StatisticsGen(
    examples=ratings_example_gen.outputs['examples'])
context.run(ratings_stats_gen, enable_cache=True)

In [ ]:
context.show(ratings_stats_gen.outputs['statistics'])

## Crie esquemas para filmes e classificações

Para um pipeline TFX, precisamos gerar um esquema de dados a partir de nosso dataset. Fazemos isto usando um [componente SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen). Isto será usado pelo [componente Transform](https://www.tensorflow.org/tfx/guide/transform) abaixo para fazer nossa engenharia de características de uma forma que seja altamente escalonável para grandes datasets e evite desvios de treinamento/serviço. Como temos dois datasets, criaremos dois componentes SchemaGen.

In [ ]:
movies_schema_gen = tfx.components.SchemaGen(
    statistics=movies_stats_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(movies_schema_gen, enable_cache=True)

In [ ]:
context.show(movies_schema_gen.outputs['schema'])

In [ ]:
ratings_schema_gen = tfx.components.SchemaGen(
    statistics=ratings_stats_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(ratings_schema_gen, enable_cache=True)

In [ ]:
context.show(ratings_schema_gen.outputs['schema'])

## Engenharia de características usando o Transform

Para um projeto estruturado e repetível de um pipeline TFX, precisaremos buscar uma abordagem escalável para a engenharia de características. Isto nos permite lidar com grandes datasets que geralmente fazem parte de muitos sistemas de recomendação e também evita desvios de treinamento/serviço. Faremos isso usando o [componente Transform](https://www.tensorflow.org/tfx/guide/transform).

O componente Transform usa um arquivo de módulo para fornecer o código do usuário para a engenharia de características que queremos fazer, portanto, nossa primeira etapa é criar esse arquivo de módulo. Como temos dois datasets, criaremos dois desses arquivos de módulo e dois componentes Transform.

Uma das coisas que nosso recomendador precisa são vocabulários para os campos `user_id` e `movie_title`. No [tutorial basic_retrieval](https://www.tensorflow.org/recommenders/examples/basic_retrieval) eles são criados com Numpy embutido, mas aqui usaremos Transform.

Observação: A magia da célula `%%writefile {_movies_transform_module_file}` abaixo cria e grava o conteúdo dessa célula num arquivo no servidor de notebooks onde este notebook está sendo executado (por exemplo, na Colab VM). Ao fazer isso fora de um notebook, você apenas criaria um arquivo Python.

In [ ]:
_movies_transform_module_file = 'movies_transform_module.py'

In [ ]:
%%writefile {_movies_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

def preprocessing_fn(inputs):
  # We only want the movie title
  return {'movie_title':inputs['movie_title']}

In [ ]:
movies_transform = tfx.components.Transform(
    examples=movies_example_gen.outputs['examples'],
    schema=movies_schema_gen.outputs['schema'],
    module_file=os.path.abspath(_movies_transform_module_file))
context.run(movies_transform, enable_cache=True)

In [ ]:
context.show(movies_transform.outputs['post_transform_schema'])

In [ ]:
inspect_examples(movies_transform, channel_name='transformed_examples')

In [ ]:
_ratings_transform_module_file = 'ratings_transform_module.py'

In [ ]:
%%writefile {_ratings_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft
import pdb

NUM_OOV_BUCKETS = 1

def preprocessing_fn(inputs):
  # We only want the user ID and the movie title, but we also need vocabularies
  # for both of them.  The vocabularies aren't features, they're only used by
  # the lookup.
  outputs = {}
  outputs['user_id'] = tft.sparse_tensor_to_dense_with_shape(inputs['user_id'], [None, 1], '-1')
  outputs['movie_title'] = tft.sparse_tensor_to_dense_with_shape(inputs['movie_title'], [None, 1], '-1')

  tft.compute_and_apply_vocabulary(
      inputs['user_id'],
      num_oov_buckets=NUM_OOV_BUCKETS,
      vocab_filename='user_id_vocab')

  tft.compute_and_apply_vocabulary(
      inputs['movie_title'],
      num_oov_buckets=NUM_OOV_BUCKETS,
      vocab_filename='movie_title_vocab')

  return outputs

In [ ]:
ratings_transform = tfx.components.Transform(
    examples=ratings_example_gen.outputs['examples'],
    schema=ratings_schema_gen.outputs['schema'],
    module_file=os.path.abspath(_ratings_transform_module_file))
context.run(ratings_transform, enable_cache=True)

In [ ]:
context.show(ratings_transform.outputs['post_transform_schema'])

In [ ]:
inspect_examples(ratings_transform, channel_name='transformed_examples')

## Implementando um modelo no TFX

No tutorial [basic_retrieval](https://www.tensorflow.org/recommenders/examples/basic_retrieval), o modelo foi criado inline no runtime do Python. Num pipeline TFX, o modelo, a métrica e a perda são definidos e treinados no arquivo de módulo para um [componente de pipeline chamado Trainer](https://www.tensorflow.org/tfx/guide/trainer). Isso torna o modelo, a métrica e a perda parte de um processo repetível que pode ser automatizado e monitorado.

### Arquitetura do modelo do TensorFlow Recommenders

Vamos construir um modelo de recuperação de duas torres. O conceito de duas torres significa que teremos uma torre de consulta computando a representação do usuário usando características do usuário, e outra torre de itens computando a representação do filme usando as características do filme. Podemos construir cada torre separadamente (nos métodos `_build_user_model()` e `_build_movie_model()` abaixo) e então combiná-las no modelo final (como na classe `MobieLensModel`). `MovieLensModel` é uma subclasse da classe base `tfrs.Model`, que agiliza a construção de modelos: tudo o que precisamos fazer é configurar os componentes no método `__init__` e implementar o método `compute_loss`, absorvendo as características brutas e retornando um valor de perda.

In [ ]:
# We're now going to create the module file for Trainer, which will include the
# code above with some modifications for TFX.

_trainer_module_file = 'trainer_module.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import Dict, List, Text

import pdb

import os
import absl
import datetime
import glob
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_recommenders as tfrs

from absl import logging
from tfx.types import artifact_utils

from tfx import v1 as tfx
from tfx_bsl.coders import example_coder
from tfx_bsl.public import tfxio

absl.logging.set_verbosity(absl.logging.INFO)

EMBEDDING_DIMENSION = 32
INPUT_FN_BATCH_SIZE = 1


def extract_str_feature(dataset, feature_name):
  np_dataset = []
  for example in dataset:
    np_example = example_coder.ExampleToNumpyDict(example.numpy())
    np_dataset.append(np_example[feature_name][0].decode())
  return tf.data.Dataset.from_tensor_slices(np_dataset)


class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model, tf_transform_output, movies_uri):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model

    movies_artifact = movies_uri.get()[0]
    input_dir = artifact_utils.get_split_uri([movies_artifact], 'train')
    movie_files = glob.glob(os.path.join(input_dir, '*'))
    movies = tf.data.TFRecordDataset(movie_files, compression_type="GZIP")
    movies_dataset = extract_str_feature(movies, 'movie_title')

    loss_metrics = tfrs.metrics.FactorizedTopK(
        candidates=movies_dataset.batch(128).map(movie_model)
        )

    self.task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=loss_metrics
        )


  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    try:
      user_embeddings = tf.squeeze(self.user_model(features['user_id']), axis=1)
      # And pick out the movie features and pass them into the movie model,
      # getting embeddings back.
      positive_movie_embeddings = self.movie_model(features['movie_title'])

      # The task computes the loss and the metrics.
      _task = self.task(user_embeddings, positive_movie_embeddings)
    except BaseException as err:
      logging.error('######## ERROR IN compute_loss:\n{}\n###############'.format(err))

    return _task


# This function will apply the same transform operation to training data
# and serving requests.
def _apply_preprocessing(raw_features, tft_layer):
  try:
    transformed_features = tft_layer(raw_features)
  except BaseException as err:
    logging.error('######## ERROR IN _apply_preprocessing:\n{}\n###############'.format(err))

  return transformed_features


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  try:
    return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size),
      tf_transform_output.transformed_metadata.schema)
  except BaseException as err:
    logging.error('######## ERROR IN _input_fn:\n{}\n###############'.format(err))

  return None


def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""
  try:
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
      """Returns the output to be used in the serving signature."""
      try:
        feature_spec = tf_transform_output.raw_feature_spec()
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        result = model(transformed_features)
      except BaseException as err:
        logging.error('######## ERROR IN serve_tf_examples_fn:\n{}\n###############'.format(err))
      return result
  except BaseException as err:
      logging.error('######## ERROR IN _get_serve_tf_examples_fn:\n{}\n###############'.format(err))

  return serve_tf_examples_fn


def _build_user_model(
    tf_transform_output: tft.TFTransformOutput, # Specific to ratings
    embedding_dimension: int = 32) -> tf.keras.Model:
  """Creates a Keras model for the query tower.

  Args:
    tf_transform_output: [tft.TFTransformOutput], the results of Transform
    embedding_dimension: [int], the dimensionality of the embedding space

  Returns:
    A keras Model.
  """
  try:
    unique_user_ids = tf_transform_output.vocabulary_by_name('user_id_vocab')
    users_vocab_str = [b.decode() for b in unique_user_ids]

    model = tf.keras.Sequential(
        [
         tf.keras.layers.StringLookup(
             vocabulary=users_vocab_str, mask_token=None),
         # We add an additional embedding to account for unknown tokens.
         tf.keras.layers.Embedding(len(users_vocab_str) + 1, embedding_dimension)
         ])
  except BaseException as err:
    logging.error('######## ERROR IN _build_user_model:\n{}\n###############'.format(err))

  return model


def _build_movie_model(
    tf_transform_output: tft.TFTransformOutput, # Specific to movies
    embedding_dimension: int = 32) -> tf.keras.Model:
  """Creates a Keras model for the candidate tower.

  Args:
    tf_transform_output: [tft.TFTransformOutput], the results of Transform
    embedding_dimension: [int], the dimensionality of the embedding space

  Returns:
    A keras Model.
  """
  try:
    unique_movie_titles = tf_transform_output.vocabulary_by_name('movie_title_vocab')
    titles_vocab_str = [b.decode() for b in unique_movie_titles]

    model = tf.keras.Sequential(
        [
         tf.keras.layers.StringLookup(
             vocabulary=titles_vocab_str, mask_token=None),
         # We add an additional embedding to account for unknown tokens.
         tf.keras.layers.Embedding(len(titles_vocab_str) + 1, embedding_dimension)
        ])
  except BaseException as err:
      logging.error('######## ERROR IN _build_movie_model:\n{}\n###############'.format(err))
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  try:
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor,
                              tf_transform_output, INPUT_FN_BATCH_SIZE)
    eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor,
                            tf_transform_output, INPUT_FN_BATCH_SIZE)

    model = MovielensModel(
        _build_user_model(tf_transform_output, EMBEDDING_DIMENSION),
        _build_movie_model(tf_transform_output, EMBEDDING_DIMENSION),
        tf_transform_output,
        fn_args.custom_config['movies']
        )

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=fn_args.model_run_dir, update_freq='batch')

    model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))
  except BaseException as err:
    logging.error('######## ERROR IN run_fn before fit:\n{}\n###############'.format(err))

  try:
    model.fit(
        train_dataset,
        epochs=fn_args.custom_config['epochs'],
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=[tensorboard_callback])
  except BaseException as err:
      logging.error('######## ERROR IN run_fn during fit:\n{}\n###############'.format(err))

  try:
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

    movies_artifact = fn_args.custom_config['movies'].get()[0]
    input_dir = artifact_utils.get_split_uri([movies_artifact], 'eval')
    movie_files = glob.glob(os.path.join(input_dir, '*'))
    movies = tf.data.TFRecordDataset(movie_files, compression_type="GZIP")

    movies_dataset = extract_str_feature(movies, 'movie_title')

    index.index_from_dataset(
      tf.data.Dataset.zip((
          movies_dataset.batch(100),
          movies_dataset.batch(100).map(model.movie_model))
      )
    )

    # Run once so that we can get the right signatures into SavedModel
    _, titles = index(tf.constant(["42"]))
    print(f"Recommendations for user 42: {titles[0, :3]}")

    signatures = {
        'serving_default':
            _get_serve_tf_examples_fn(index,
                                      tf_transform_output).get_concrete_function(
                                          tf.TensorSpec(
                                              shape=[None],
                                              dtype=tf.string,
                                              name='examples')),
    }
    index.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

  except BaseException as err:
      logging.error('######## ERROR IN run_fn during export:\n{}\n###############'.format(err))

## Treinando o modelo

Após definir o modelo, podemos executar o [componente Trainer](https://www.tensorflow.org/tfx/guide/trainer) para fazer o treinamento do modelo.

In [ ]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_trainer_module_file),
    examples=ratings_transform.outputs['transformed_examples'],
    transform_graph=ratings_transform.outputs['transform_graph'],
    schema=ratings_transform.outputs['post_transform_schema'],
    train_args=tfx.proto.TrainArgs(num_steps=500),
    eval_args=tfx.proto.EvalArgs(num_steps=10),
    custom_config={
        'epochs':5,
        'movies':movies_transform.outputs['transformed_examples'],
        'movie_schema':movies_transform.outputs['post_transform_schema'],
        'ratings':ratings_transform.outputs['transformed_examples'],
        'ratings_schema':ratings_transform.outputs['post_transform_schema']
        })

context.run(trainer, enable_cache=False)

## Exportando o modelo

Após treinar o modelo, podemos usar o [componente Pusher](https://www.tensorflow.org/tfx/guide/pusher) para exportar o modelo.

In [ ]:
_serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model/tfrs_retrieval')

pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher, enable_cache=True)

## Faça previsões

Agora que temos um modelo, carregamos ele novamente e fazemos previsões.

In [ ]:
loaded = tf.saved_model.load(pusher.outputs['pushed_model'].get()[0].uri)
scores, titles = loaded(["42"])

print(f"Recommendations: {titles[0][:3]}")

## Próximos passos

Neste tutorial, você aprendeu como implementar um modelo de recuperação com recomendadores TensorFlow e TFX. Para ir além do que foi apresentado aqui, dê uma olhada no tutorial [Ranking TFRS com o TFX](https://www.tensorflow.org/recommenders/examples/ranking_tfx).